In [1]:
import re
import bs4
import os
import csv

In [2]:
data_path = r"C:\Users\billy\Desktop\py\ffnorma\data_src"

# bazy a(aktywne), w(wycofane)
db_a = []
db_w = []

# baza A
for filename in os.listdir(data_path):
    if filename.endswith("A.html"): 

        with open(f"{data_path}\\{filename}", encoding="utf-8") as website:
            soup = bs4.BeautifulSoup(website, "html.parser")

        blocks = soup.find_all("div", class_="p4s-search-results-values")    

        for b in blocks:
            tytul = None
            numer = None
            zastepuje = []

            if len(b.find_all("span", string=re.compile("Tytuł normy"))) == 0:
                tytul = ""
            else:
                tytul = str(
                            b.find_all(
                                "span", string=re.compile("Tytuł normy")
                            )[0].find_next("a").string
                        ).replace('\t', '').replace('\n', '')
                
                numer = "PN"+ str(
                                b.find_all(
                                    "span", class_="highlighted-search-term"
                                )[0].next_sibling
                            ).replace('\t', '').replace('\n', '')

                if len(b.find_all(string=re.compile("Zastępuje"))) != 0:
                    zastapione_normy = b.find_all(
                                            string=re.compile("Zastępuje")
                                        )[0].parent.parent.next_sibling.find_next("ul").find_all("a")
                    for z in zastapione_normy:
                        try:
                            z = z.contents[0].replace('\t', '').replace('\n', '')
                        except IndexError:
                            z = ''
                        zastepuje.append(z)

            db_a.append((tytul, numer, zastepuje))

db_filtered = list(filter(lambda x: x[0] is not '', db_a))


# baza W
for filename in os.listdir(data_path):
    if filename.endswith("W.html"):
        with open(f"{data_path}\\{filename}", encoding="utf-8") as website:
            soup_w = bs4.BeautifulSoup(website, "html.parser")

        blocks_w = soup_w.find_all("div", class_="p4s-search-results-values")    

        for b in blocks_w:
            numer = ""
            zastepuje = []

            if len(b.find_all("span", string=re.compile("Tytuł normy"))) == 0:
                continue

            else:
                numer = "PN"+ str(b.find_all("span", class_="highlighted-search-term")[0].next_sibling).replace('\t', '').replace('\n', '')

                if len(b.find_all(string=re.compile("Zastępuje"))) != 0:
                            zastapione_normy = b.find_all(string=re.compile("Zastępuje"))[0].parent.parent.next_sibling.find_next("ul").find_all("a")
                            for z in zastapione_normy:
                                try:
                                    z = z.contents[0].replace('\t', '').replace('\n', '')
                                except IndexError:
                                    z = ''
                                zastepuje.append(z)

            db_w.append((numer, zastepuje))


In [4]:
db_w_tab = []

for i in range(len(db_w)):
    temp = [db_w[i][0]]
    temp.extend(db_w[i][1])
    db_w_tab.append(set(temp))
    
for s in range(len(db_w_tab)):
    for i in range(s+1, len(db_w_tab)):
        if len(db_w_tab[s].intersection(db_w_tab[i])) > 0:
            db_w_tab[s].union(db_w_tab[i])
            db_w_tab[i] = set()
            
w_reduced = []
for e in db_w_tab:
    if e != set():
        w_reduced.append(e)

# Dana jest lista tupli aktualnych norm z normami zastępującymi ('Tytul', 'Numer aktualnej', [lista numerów nieaktualnych])
# oraz lista zbiorów {'nieaktualnych, powiązanych ze sobą numerów norm', ... }
merge_db = []

for element in db_filtered:
    merged_olds = set(element[2])
    
    for old_norm in w_reduced:
        if len(merged_olds.intersection(old_norm)) > 0:
            merged_olds.union(old_norm)
                
    merge_db.append((element[0], element[1], merged_olds))


In [5]:
# Zapisywanie do CSV

with open("db.csv", "w", encoding="utf-8", newline="") as dbfile:
    writer = csv.writer(dbfile, delimiter=',')
    writer.writerows(merge_db)

In [6]:
# Wczytanie CSV do listy
# csv_read = []

# with open("db.csv", "r", encoding="utf-8", newline="") as readdb:
#     reader = csv.reader(readdb, delimiter=',')
#     for row in reader:
#         csv_read.append((row[0], row[1], eval(row[2])))